In [1]:
#from two_stage_rag_pipeline import *
#from rag_pipeline import *
from test_pipeline import *
import os
import pandas as pd
from uuid import uuid4
from tqdm.notebook import tqdm
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_precision,
    context_recall,
)
# 최신 버전 기준 (ragas >= 0.1.2)
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from ragas import EvaluationDataset
from datasets import Dataset as HFDataset
# 초기 세팅
vectorstore = initialize_pinecone()
chain = rag_chain(vectorstore)

모델 호출 중...


In [2]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [3]:
import os
from dotenv import load_dotenv
from datasets import Dataset


def main():
    load_dotenv()

    # 저장 폴더 준비
    os.makedirs("ragas_test_data", exist_ok=True)

    # 질문 리스트
    questions = [
        "월요일 7, 8교시에 들을 수 있는 교양 있으면 알려줘.",
        "글스산 졸업하면 뭐해?",
        "졸업하기 전에 무슨 영어성적 제출해야 된다고 했던거 같은데 그건 뭐야?",
        "후기 이중에 대해 설명해줘.",
        "김정환 교수님 연구실 어디야?",
        "컴퓨팅사고를 AI융합전공 이중과목으로 변경해야되는데 이수구분변경 신청 어떻게 해?",
        "감기로 병원다녀왔는데 진료확인서로 유고결석 가능해?",
        ("나 정보통신공학과인데 혹시 졸업시험 대신 졸업논문을 선택해도 될까? "
         "논문을 쓰게 되면 머신러닝 관련해서 작성하고 싶은데 이와 관련해서 상담할 수 있는 교수님이 계시면 이메일 좀 알려줘."),
        "일통 FLEX 시험 점수 제출 기간 알고 싶어.",
        "글스산 세부전공 신청해야되는데, 신청기간이랑 방법 알 수 있어? 그리고 결과는 언제나와?"
    ]

    # Ground‐truth 리스트
    reference = [
        ["월요일 7,8교시에 들을 수 있는 교양 과목에는 '인간과 사회', '대중문화 예술의 이해', '비즈니스협상' 등이 있습니다."],
        ["글로벌스포츠산업학부를 졸업하면 주로 다음과 같은 분야에서 활동하게 됩니다:\n"
         "1. 스포츠매니지먼트 및 이벤트 기획\n"
         "2. 스포츠 마케팅 및 홍보\n"
         "3. 스포츠 데이터 분석가\n"
         "4. 스포츠 미디어\n"
        "이 외에도 졸업 후에는 대학원 진학이나 관련 분야의 연구를 진행할 수 있는 기회도 있습니다."
        ],
        ["졸업하기 전에 제출해야 하는 영어 성적은 외국어 인증을 위한 것으로, 다음과 같은 기준을 충족해야 합니다:\n\n1." 
        "**인증 시험 성적**: 제출 시점 기준 유효기간(2년) 이내의 인증 시험 성적이 필요합니다. 인정되는 시험은 다음과 같습니다:\n"   
        "- TOEIC\n   - TOEFL\n   - IELTS\n   - FLEX 외국어 능력 시험 등\n\n2." 
        "**대체 과정 수료증**: 인증 시험 성적 대신 외국어 연수 평가원 또는 외국어 교육센터에서 제공하는 대체 과정을 이수한 경우," 
        "해당 수료증을 제출할 수 있습니다.\n\n3. **제출 마감일**: 졸업하고자 하는 최종 학기 말까지 반드시 제출해야 하며," 
        "재학 기간 동안 1회 외국어 인증 신청서를 제출할 수 있습니다.\n\n"
        "정확한 기준과 제출 방법은 학사종합지원센터에 문의하시거나, 해당 과목의 담당 교수님께 확인하시기 바랍니다."],
         ["후기 이중전공은 부전공 이수자가 정규 8학기 내에 제1전공 졸업학점(부전공 21학점 포함)을 모두 취득한 후, "
         "1~2학기 추가 수학하여 해당 부전공의 이중전공화에 필요한 나머지 학점을 취득함으로써, "
         "제1전공과 이중전공 학위를 동시에 취득하는 제도입니다.\n"
        "신청은 매년 4월에 한 번만 가능하며, 신청 자격은 현재 7~8학기 이수 중인 학생으로, 부전공을 이수 중인 경우에 해당합니다.\n" 
        "추가학기를 등록해야 하며, 등록금은 후기 이중전공 학과의 등록금으로 납부해야 합니다. 더 궁금하신 점이 있으시면 말씀해 주시기 바랍니다."],
        ["김정환 교수님의 연구실은 어문관 435호에 위치해 있습니다. 추가로 궁금하신 점이 있으시면 언제든지 말씀해 주세요."],
        ["컴퓨팅사고를 AI융합전공 이중과목으로 변경하시려면, 이수구분 변경 신청서를 작성하신 후," 
        "해당 전공 소속 단과대학의 담당 선생님에게 이메일로 제출하시거나 직접 방문하여 제출하셔야 합니다.\n"
        "신청은 졸업학기 졸업사정일 전까지 가능하며, 제출 후 3~4일 이내에 종합정보시스템에서 변경 결과를 확인하실 수 있습니다.\n" 
        "만약 변경이 되지 않았거나 문제가 발생한 경우, 즉시 담당 선생님에게 연락하여 확인하셔야 합니다."
        "자세한 사항은 학사종합지원센터의 공지사항을 참고하시기 바랍니다. 도움이 필요하시면 언제든지 문의해 주세요."],
        ["감기로 병원에 다녀오신 경우, 진료확인서로 유고결석을 인정받기 위해서는 몇 가지 조건이 있습니다."
         "일반적으로 의원에서 발급받은 서류는 유고결석 인정에 해당되지 않으며, 병원급 이상의 의료기관에서 발급된 증빙서류가 필요합니다."
         "따라서, 감기로 병원급 의료기관에서 진료를 받으셨다면 해당 증빙서류를 제출하시면 유고결석이 가능할 수 있습니다.\n"
        "정확한 사항은 담당 교수님께 문의하시고, 필요한 서류를 제출하시는 것이 좋습니다. 추가적인 질문이 있으시면 언제든지 말씀해 주시기 바랍니다."],
        ["졸업시험 대신 졸업논문을 선택하실 수 있습니다."
         "졸업논문을 작성하실 경우, 머신러닝 관련 주제로 진행하고 싶으시다면, 관련 상담을 위해 한희일 교수님께 연락하실 수 있습니다\n"
         "한희일 교수님의 이메일은 hihahn@hufs.ac.kr 입니다."],
        ["졸업시험 점수 제출 기간은 공지사항에 따라 각 학과 사무실로 문의해야 합니다. 추가적인 질문이 있으시면 말씀해주세요."],
        ["글로벌스포츠산업학부의 세부전공 신청에 대한 정보는 다음과 같습니다.\n"
         "신청기간: 2025년 5월 26일(월) 10:00부터 5월 30일(금) 16:00까지입니다. 신청기간 이후에는 추가 신청이나 변경이 절대 불가하니 유의하시기 바랍니다.\n"
         "신청방법: 종합정보시스템에 접속하셔서 전공/교직관리 메뉴에서 1전공 신청을 하시면 됩니다.\n"
         "배정결과: 배정 확정은 2025년 7월 중에 이루어질 예정입니다. 성적 반영이 완료된 후에 결과를 확인하실 수 있습니다.\n"
         "추가적인 문의사항이 있으시면 학사종합지원센터에 연락하시기 바랍니다."]
    ]

    # RAG 추론 및 컨텍스트 수집
    answers = []
    contexts = []
    #session_id = "test-session"  # 또는 str(uuid4()) 사용
    
    for q in questions:
        result = chain.invoke(q)
    
        answers.append(result)  # ⬅️ 문자열만 저장
        docs = retriever.invoke(q)
        contexts.append([doc.page_content for doc in docs])
    # Dataset 생성
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "reference": reference
    }
    dataset = Dataset.from_dict(data)


    dataset.save_to_disk("ragas_test_data")

    print("Dataset saved to data")
    print(dataset)

if __name__ == "__main__":
    main()

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset saved to data
Dataset({
    features: ['question', 'answer', 'contexts', 'reference'],
    num_rows: 10
})


In [4]:
from datasets import load_from_disk
from ragas import evaluate
from ragas.metrics import (
    context_precision,
    context_recall,
    faithfulness,
    answer_relevancy,
)

def main():
    # 저장된 dataset 불러오기
    dataset = load_from_disk("ragas_test_data")

    def convert_reference(example):
        ref = example["reference"]
        if isinstance(ref, list):
            example["reference"] = ref[0]
        return example
    
    # 3. map으로 전체에 적용
    dataset = dataset.map(convert_reference)

    # 평가 실행
    result = evaluate(
        dataset=dataset,
        metrics=[
            context_precision,
            context_recall,
            faithfulness,
            answer_relevancy,
        ],
    )

    # 결과 확인 및 저장
    print(result)
    df = result.to_pandas()
    df.to_csv("data/evaluation_results_base.csv", index=False)
    print("Evaluation results saved to evaluation_results.csv")

if __name__ == "__main__":
    main()

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

{'context_precision': 0.6650, 'context_recall': 0.3883, 'faithfulness': 0.6625, 'answer_relevancy': 0.5045}
Evaluation results saved to evaluation_results.csv


In [9]:
import pandas as pd
df = pd.read_csv('data/evaluation_results_base.csv')
df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,"월요일 7, 8교시에 들을 수 있는 교양 있으면 알려줘.","['[언어학연구방법론] 4학년 대상 / 교수: 이해윤 / 월요일 7,8교시 / 강의...","죄송하지만 월요일 7, 8교시에 들을 수 있는 교양 과목은 [신화와종교] 밖에 없습니다.","월요일 7,8교시에 들을 수 있는 교양 과목에는 '인간과 사회', '대중문화 예술의...",0.000000,0.000000,0.000000,0.796662
1,글스산 졸업하면 뭐해?,['3학년 통번역을 중점적으로 다루며 졸업시험을 위한 FLEX 준비도 시\n작하게...,"졸업 후에는 정부 및 공공기관, 기업, 학계, 창업 등 다양한 분야로 진출할 수 있...",글로벌스포츠산업학부를 졸업하면 주로 다음과 같은 분야에서 활동하게 됩니다:\n1. ...,0.500000,0.000000,0.750000,0.000000
2,졸업하기 전에 무슨 영어성적 제출해야 된다고 했던거 같은데 그건 뭐야?,"['| 제출마감일 | 재학기간 동안\xa01회,\xa0외국어 인증 신청서를 제출하고...","졸업 전에 제출해야 하는 영어 성적은 외국어 인증 신청서와 해당 시험의 성적이며, ...","졸업하기 전에 제출해야 하는 영어 성적은 외국어 인증을 위한 것으로, 다음과 같은 ...",0.866667,0.500000,0.500000,0.853168
3,후기 이중에 대해 설명해줘.,['을까요?\nA3. 전쟁 이후 우크라이나어의 위상은 높아지고 사용 빈도는 증가할...,후기 이중에 대해 설명하겠습니다. 해당 문서에는 후기 이중에 대한 내용이 없습니다....,후기 이중전공은 부전공 이수자가 정규 8학기 내에 제1전공 졸업학점(부전공 21학점...,0.000000,0.750000,0.333333,0.000000
4,김정환 교수님 연구실 어디야?,['김정환 교수님은 루마니아학과 학과 소속입니다. 교수 직위를 맡고 있습니다. 이메...,김정환 교수님의 연구실은 어문관 435호에 위치해 있습니다.,김정환 교수님의 연구실은 어문관 435호에 위치해 있습니다. 추가로 궁금하신 점이 ...,1.000000,0.500000,1.000000,0.906377
5,컴퓨팅사고를 AI융합전공 이중과목으로 변경해야되는데 이수구분변경 신청 어떻게 해?,['[공통] 2025-1학기 이중전공 변경 신청 일정 공고(비즈니스 & AI 융합전...,이중전공 변경 신청은 기존 AI 융합전공 및 세부트랙에 진입한 인원만 변경 신청이 ...,"컴퓨팅사고를 AI융합전공 이중과목으로 변경하시려면, 이수구분 변경 신청서를 작성하신...",1.000000,0.500000,0.875000,0.830813
6,감기로 병원다녀왔는데 진료확인서로 유고결석 가능해?,['## [paragraph] ##\n나. 처리절차\n\n## [paragraph]...,"네, 감기로 병원 다녀온 경우 진료확인서로 유고결석이 가능하지 않아요. 질병 및 상...","감기로 병원에 다녀오신 경우, 진료확인서로 유고결석을 인정받기 위해서는 몇 가지 조...",1.000000,0.800000,0.666667,0.000000
7,나 정보통신공학과인데 혹시 졸업시험 대신 졸업논문을 선택해도 될까? 논문을 쓰게 되...,['교내해커톤 등 다양한 공모전에 팀을 이루어서 프로젝트를 수행하고 \n발표하는 경...,"네, 정보통신공학과에서는 졸업 논문을 선택할 수 있습니다. 논문을 쓰고자 하는데 상...","졸업시험 대신 졸업논문을 선택하실 수 있습니다.졸업논문을 작성하실 경우, 머신러닝 ...",0.916667,0.333333,0.500000,0.834685
8,일통 FLEX 시험 점수 제출 기간 알고 싶어.,"['6. FLEX 대체 ( 정기시험 : 영어 , 중국어 , 일본어 , 프랑스어 , ...",일통 FLEX 시험의 제출기간은 해당학과 사무실에 문의하셔야 합니다. 해당 기간은 ...,졸업시험 점수 제출 기간은 공지사항에 따라 각 학과 사무실로 문의해야 합니다. 추가...,1.000000,0.500000,1.000000,0.000000
9,"글스산 세부전공 신청해야되는데, 신청기간이랑 방법 알 수 있어? 그리고 결과는 언제나와?","['[글로벌] 이수구분변경 신청 방법 안내\n\n다. 이수구분 신청 전, 전공교류교...","신청기간은 2025년 1월 22일(수) 10시부터 1월 23일(목) 14시까지이며,...",글로벌스포츠산업학부의 세부전공 신청에 대한 정보는 다음과 같습니다.\n신청기간: 2...,0.366667,0.000000,1.000000,0.823345


In [10]:
# 보기 좋게 출력하는 함수
def print_ragas_eval(df):
    for i, row in df.iterrows():
        print(f"--- Row {i} ---")
        print(f"[User Input]\n{row['user_input']}\n")
        print(f"[Response]\n{row['response']}\n")
        print(f"[Reference]\n{row['reference']}\n")
        print(f"[Faithfulness]: {row['faithfulness']:.3f}")
        print(f"[Answer Relevancy]: {row['answer_relevancy']:.3f}")
        print("=" * 40)

# 사용 예시 (상위 5개만 출력)
print_ragas_eval(df)

--- Row 0 ---
[User Input]
월요일 7, 8교시에 들을 수 있는 교양 있으면 알려줘.

[Response]
죄송하지만 월요일 7, 8교시에 들을 수 있는 교양 과목은 [신화와종교] 밖에 없습니다.

[Reference]
월요일 7,8교시에 들을 수 있는 교양 과목에는 '인간과 사회', '대중문화 예술의 이해', '비즈니스협상' 등이 있습니다.

[Faithfulness]: 0.000
[Answer Relevancy]: 0.797
--- Row 1 ---
[User Input]
글스산 졸업하면 뭐해?

[Response]
졸업 후에는 정부 및 공공기관, 기업, 학계, 창업 등 다양한 분야로 진출할 수 있어요. 한국외대 그리스·불가리아학과 졸업생은 대사관, 대기업, 방송사, 항공사, 언론사, 광고·PR사, IT 기업 등에서 활동할 수 있습니다. 또한 국내외 연구기관 연구원이나 대학교수로 진로를 선택할 수도 있어요. 자세한 정보는 학과 전공가이드북을 참고해주세요.

[Reference]
글로벌스포츠산업학부를 졸업하면 주로 다음과 같은 분야에서 활동하게 됩니다:
1. 스포츠매니지먼트 및 이벤트 기획
2. 스포츠 마케팅 및 홍보
3. 스포츠 데이터 분석가
4. 스포츠 미디어
이 외에도 졸업 후에는 대학원 진학이나 관련 분야의 연구를 진행할 수 있는 기회도 있습니다.

[Faithfulness]: 0.750
[Answer Relevancy]: 0.000
--- Row 2 ---
[User Input]
졸업하기 전에 무슨 영어성적 제출해야 된다고 했던거 같은데 그건 뭐야?

[Response]
졸업 전에 제출해야 하는 영어 성적은 외국어 인증 신청서와 해당 시험의 성적이며, 졸업 예정자는 졸업하고자 하는 최종 학기 말까지 반드시 제출해야 합니다.

[Reference]
졸업하기 전에 제출해야 하는 영어 성적은 외국어 인증을 위한 것으로, 다음과 같은 기준을 충족해야 합니다:

1.**인증 시험 성적**: 제출 시점 기준 유효기간(2년) 이내의 인증 시험

In [28]:
print(df['retrieved_contexts'][9])

['[글로벌] 이수구분변경 신청 방법 안내\n\n다. 이수구분 신청 전, 전공교류교과목을 수강편람에서 확인하고 문의 사항이 있을 경우 신청전 미리 상담 (수강편람 확인 클릭)\n\n라. 신청 후 3~4일 (평일 기준) 이후\xa0종합정보시스템에서 변경결과를 반드시 확인하고 변경이 안된 경우, 담당자에게 연락하여 반드시 확인', '[글로벌] 이수구분변경 신청 방법 안내\n\n다. 이수구분 신청 전, 전공교류교과목을 수강편람에서 확인하고 문의 사항이 있을 경우 신청전 미리 상담 (수강편람 확인 클릭)\n\n라. 신청 후 3~4일 (평일 기준) 이후\xa0종합정보시스템에서 변경결과를 반드시 확인하고 변경이 안된 경우, 담당자에게 연락하여 반드시 확인', '다. 이수구분 신청 전, 전공교류교과목을 수강편람에서 확인하고 문의 사항이 있을 경우 신청전 미리 상담 (수강편람 확인 클릭)\n\n라. 신청 후 3~4일 (평일 기준) 이후\xa0종합정보시스템에서 변경결과를 반드시 확인하고 변경이 안된 경우, 담당자에게 연락하여 반드시 확인', '▶ ( 글로벌 ) hufsdbl@hufs.ac.kr 로 이메일 제출\n\n2. 일 정\n\n가 . 신청 기간 : 2025 년 1 월 22 일 ( 수 ) 10 시 ~ 1 월 23 일 ( 목 ) 14 시\n\n※ 안내된 기간에만 운영하므로 일정 준수할 것 ( 이후 별도 운영 계획 없음 )\n\n나 . 결과 확인 : 2025 년 1 월 24 일 ( 금 ) 14 시 , 종합정보시스템\n\n3. 신청 방법 : 종합정보시스템 → 전공 / 교직관리 ( ※ 가 . / 나 . 중 하나의 방법으로 신청 )\n\n가 . 부전공 신청 : 전공심화 + 부전공 또는 부전공 신청\n\n1) 신청 시 기존에 이수 중인 이중 / 부전공 은 자동 취소 됨 ( ※ 별도 취소 절차 없음 )\n\n2) “4. 유의 사항 ” 의 다 . 참고 하여 부전공 신청으로 인한 이수구분 변경 신청\n\n※ 이번 학기 졸업예정자 (4 학년 수료 , 졸업대기 포함 ) 는 1 월 초 까지\n

In [6]:
df1 = df.mean(numeric_only=True).reset_index()

In [7]:
# 1. 평균값 구하고 전치
df1 = df.mean(numeric_only=True, skipna=True).reset_index()
df1 = df1.T

# 2. 첫 번째 행을 컬럼명으로 설정
df1.columns = df1.iloc[0]

# 3. 첫 번째 행(이전 헤더로 썼던 행) 제거
df1 = df1.drop(df1.index[0]).reset_index(drop=True)

# 1. 평균값 구하고 전치
df1 = df.mean(numeric_only=True, skipna=True).reset_index()
df1 = df1.T

# 2. 첫 번째 행을 컬럼명으로 설정
df1.columns = df1.iloc[0]

# 3. 첫 번째 행(이전 헤더로 썼던 행) 제거
df1 = df1.drop(df1.index[0]).reset_index(drop=True)

# 4. 모델 컬럼 추가 및 인덱스 삭제

df1['model'] = 'base'

df1

index,context_precision,context_recall,faithfulness,answer_relevancy,model
0,0.665,0.388333,0.6625,0.504505,base


In [8]:
df1.to_csv('data/base.csv',encoding='utf-8',index=False)